<a href="https://colab.research.google.com/github/Swicano/IntroML_NBs/blob/master/LSTM_Math_ASCII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras import *
from keras.models import Sequential
from keras.layers import Dense, LSTM

Using TensorFlow backend.


In [0]:
from numpy import *
import random
import matplotlib.pyplot as plt
%matplotlib inline

### Define a batch function and all that

In [0]:
# a function to generate arbitrary math problems

def Gen_expr(terms=2, val_range=[1,10], trunc='{:.1f}', ops=['+','-','*','/'], rec_max=2, rec_ops = ['', 'sin','exp'], rec_prob = 0.33 ):
  # rec refers to recursion, which is how i originally envisioned parentheticals and functions to be built
  # but flattening arbitrarily recursive lists seemed like more work than just making flat ones to begin
  rec_level = 0
  expr_list = []
  
  term_list = [ trunc.format(random.uniform(val_range[0],val_range[1])) for i in range(terms) ]
  ops_list = random.choices(ops,k=terms-1)
  final = term_list.pop()
  
  # if someone puts in an empty rec list 
  if len(rec_ops)==0: 
    rec_ops = ['']
    
  while term_list:
    if (rec_level<rec_max and random.random()<rec_prob):
      rec_level+=1
      expr_list.append( random.choice(rec_ops)+'(')
    expr_list.append(term_list.pop())
    if rec_level and random.random()<0.4:
      rec_level-=1
      expr_list.append(')')
    expr_list.append(ops_list.pop())  
  
  # then finish off by adding the last number, and closing any open parens
  expr_list.append(final)
  while rec_level:
    rec_level-=1
    expr_list.append(')')
  expr_str = ''.join(expr_list)
  
  return expr_str

In [163]:
#ignore this
inputlen = 10
informat = '{:>'+str(inputlen)+'}'
bob = Gen_expr(val_range=[1,5],ops=['+','-'],trunc='{:.0f}' ,rec_max=0)
bobstr = informat.format(bob+'=')

arr1 =fromstring(bobstr, int8)-48
print(bobstr, arr1)

      2-3= [-16 -16 -16 -16 -16 -16   2  -3   3  13]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


In [0]:
def make_batch(batchsize = 1000):
  terms = 2
  term_range = [1,9]
  truncformat = '{:.0f}'  # the truncation format  '{:.1f}', {:.0f} for int
  ops=['+']           # the potential operations supported
  recs=['']               # the potential functions ('' = parens)
  rec_prob = 0.33         # the probability of using one of the functions
  max_term_len = max(len(truncformat.format(term_range[0])),len(truncformat.format(term_range[1])))
  max_ops_len  = max([len(i) for i in ops]) 
  max_rec_len  = max([len(i) for i in recs]) 
  
  inputlen =(max_term_len+2)*terms # the +2 takes care of if each term somehow gets parens
  inputlen+=max_ops_len*(terms-1)  # for the longest possible operations in each slot
  inputlen+=int(max_rec_len*(rec_prob)) # for the average number of function cals
  inputlen+=1                      # for the equals sign
  outputlen=max_term_len+1        # assume the answer is only 1 digit larger (this is bad)
  #print(inputlen, outputlen)
  
  # this does a raw input encoding, not a one-hot style encoding
  informat = '{:>'+str(inputlen)+'}'
  outformat = '{:>'+str(outputlen)+'}'
  inbatch  = zeros([batchsize, inputlen,1])
  outbatch = zeros([batchsize, inputlen,outputlen])
  for i in range(batchsize):
    # a hack since it could divide by 0
    test = ''
    outtest = ''
    while True:
      try:
        test = Gen_expr(terms=terms, val_range=term_range, trunc=truncformat, ops=ops, rec_max=2, rec_ops = recs, rec_prob =rec_prob)
        outtest = truncformat.format(eval(test))
      except ZeroDivisionError:
        #print(test, 'err!')
        continue
      else:
        #do stuff?
        break
    inbatch2 = fromstring(informat.format(test+'='),dtype=uint8).reshape((inputlen,1))
    inbatch[i,] = inbatch2
    outbatch2 = fromstring(outformat.format(outtest),dtype=uint8).reshape((1,outputlen))
    outbatch[i,inputlen-1,] = outbatch2
  #print(inbatch, outbatch)
  return inbatch, outbatch
  #return test, outtest
    
    
    
    
    
  

In [7]:
#Generate our validation set, of which we will use number 1 to visualize
Val_in, Val_out = make_batch(1000)
print(Val_in[0], Val_out[0]) # subtract 48 to get the number value, 32='', 43='+', 61='=', 40='(', 41=')'

[[32.]
 [32.]
 [32.]
 [32.]
 [57.]
 [43.]
 [54.]
 [61.]] [[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [49. 53.]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [0]:
'{:.0f}'.format(3/4.6)
truncfunc = '{:.1f}'
truncfunc.format(3/4.6)

'0.7'

### Make the net

In [0]:
net=Sequential()
net.add(LSTM(5, batch_input_shape=[None,8,1], return_sequences=True))
net.add(LSTM(2, return_sequences=True))
net.add(Dense(2, activation = 'linear'))
net.compile(loss='mean_squared_error',optimizer='adam', metrics=['accuracy'])

In [33]:
epochs = 2000
test_output = zeros([8,2,epochs])
for k in range(epochs):
  train_in, train_out = make_batch(3000)  # training data
  net.train_on_batch(train_in, train_out)
  test_output[:,:,k] = net.predict_on_batch(Val_in)[0,:,:]
  print("\r epoch: ",k," deviation: ", sum((test_output[:,:,k]-Val_out[0,:,:])**2),"    ")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


 epoch:  0  deviation:  1975.1224787571664     
 epoch:  1  deviation:  1988.9185163053635     
 epoch:  2  deviation:  1970.776191543312     
 epoch:  3  deviation:  1942.403393328732     
 epoch:  4  deviation:  1957.4482646279503     
 epoch:  5  deviation:  2006.8462289407626     
 epoch:  6  deviation:  2002.8874992833262     
 epoch:  7  deviation:  1944.5002035567604     
 epoch:  8  deviation:  1947.3888583980706     
 epoch:  9  deviation:  2003.0236271564045     
 epoch:  10  deviation:  1993.5502046806732     
 epoch:  11  deviation:  1957.192508685439     
 epoch:  12  deviation:  1948.1676505720807     
 epoch:  13  deviation:  1961.9779744603402     
 epoch:  14  deviation:  1964.3738550823336     
 epoch:  15  deviation:  1935.914796398043     
 epoch:  16  deviation:  1957.8913847658368     
 epoch:  17  deviation:  1991.1006449602546     
 epoch:  18  deviation:  1977.8827343179337     
 epoch:  19  deviation:  1947.3673957150068     
 epoch:  20  deviation:  1955.4158

In [11]:
test3 , test4 = make_batch(1)
print(test3,test4)
shape(test3), shape(test4)
print(shape(Val_out))

[[[32.]
  [32.]
  [40.]
  [52.]
  [41.]
  [43.]
  [55.]
  [61.]]] [[[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [49. 49.]]]
(1000, 8, 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [15]:
bob = net.predict_on_batch(Val_in)
print(shape(bob))

(1000, 8, 2)


In [34]:
print(Val_out[0])
test_output[:,:,1999]

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [49. 53.]]


array([[-1.13105774e-03, -6.98165894e-02],
       [-1.09859467e-01,  6.37340546e-02],
       [ 4.87258911e-01,  7.46109009e-01],
       [ 3.58690262e-01,  6.07580185e-01],
       [ 3.41224670e-01,  5.88762283e-01],
       [ 3.38865280e-01,  5.86219788e-01],
       [ 8.86823177e+00,  9.77635765e+00],
       [ 2.27547703e+01,  2.47386875e+01]])